# QLD Data Pipeline

This data pipeline extracts and prepares the QLD donations, lobbyist, and their client data. \
The outputs of this pipeline includes **qldlobbyist.csv, qldlbClient.csv, qld_donations.csv, and qld_networkg.csv** \
which can be found in the 'data' folder in this repository.

## Install and load Packages

### Prerequisites

- A **cloned government_transparency public Github repository on local directory** where this notebook file is excuted from.
- A correct version of chromedriver is required on the working directory. \
Please install the version that matches your browser.
https://chromedriver.chromium.org/downloads

- Ghostscript and ActiveTcl are required to be installed on the machine. \
See instructions on:
https://camelot-py.readthedocs.io/en/master/user/install-deps.html \
Re-boot maybe required following a new installation.

### Load Common Libraries

In [1]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML
try:
    import scrapy # scrape webpage
except:
    !pip install scrapy
    import scrapy
from scrapy.crawler import CrawlerProcess
# text cleaning
import re
# Settings for notebook
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# Show Python version
import platform
print("Installed Python version" , platform.python_version())

Installed Python version 3.8.8


## Extract & Transform
Process all files into CSV and load CSVs

### Extract Lobbyist Data

The Register of Lobbyists is a list of professional lobbyists who wish to lobby Government representatives. \
Extracting the QLD lobbysit data from https://lobbyists.integrity.qld.gov.au/register-details/list-companies.aspx

#### middlewares

In [2]:
from scrapy import signals

# useful for handling different item types with a single interface
from itemadapter import is_item, ItemAdapter


class QldlobbyistSpiderMiddleware:
    # Not all methods need to be defined. If a method is not defined,
    # scrapy acts as if the spider middleware does not modify the
    # passed objects.

    @classmethod
    def from_crawler(cls, crawler):
        # This method is used by Scrapy to create your spiders.
        s = cls()
        crawler.signals.connect(s.spider_opened, signal=signals.spider_opened)
        return s

    def process_spider_input(self, response, spider):
        # Called for each response that goes through the spider
        # middleware and into the spider.

        # Should return None or raise an exception.
        return None

    def process_spider_output(self, response, result, spider):
        # Called with the results returned from the Spider, after
        # it has processed the response.

        # Must return an iterable of Request, or item objects.
        for i in result:
            yield i

    def process_spider_exception(self, response, exception, spider):
        # Called when a spider or process_spider_input() method
        # (from other spider middleware) raises an exception.

        # Should return either None or an iterable of Request or item objects.
        pass

    def process_start_requests(self, start_requests, spider):
        # Called with the start requests of the spider, and works
        # similarly to the process_spider_output() method, except
        # that it doesn’t have a response associated.

        # Must return only requests (not items).
        for r in start_requests:
            yield r

    def spider_opened(self, spider):
        spider.logger.info('Spider opened: %s' % spider.name)


class QldlobbyistDownloaderMiddleware:
    # Not all methods need to be defined. If a method is not defined,
    # scrapy acts as if the downloader middleware does not modify the
    # passed objects.

    @classmethod
    def from_crawler(cls, crawler):
        # This method is used by Scrapy to create your spiders.
        s = cls()
        crawler.signals.connect(s.spider_opened, signal=signals.spider_opened)
        return s

    def process_request(self, request, spider):
        # Called for each request that goes through the downloader
        # middleware.

        # Must either:
        # - return None: continue processing this request
        # - or return a Response object
        # - or return a Request object
        # - or raise IgnoreRequest: process_exception() methods of
        #   installed downloader middleware will be called
        return None

    def process_response(self, request, response, spider):
        # Called with the response returned from the downloader.

        # Must either;
        # - return a Response object
        # - return a Request object
        # - or raise IgnoreRequest
        return response

    def process_exception(self, request, exception, spider):
        # Called when a download handler or a process_request()
        # (from other downloader middleware) raises an exception.

        # Must either:
        # - return None: continue processing this exception
        # - return a Response object: stops process_exception() chain
        # - return a Request object: stops process_exception() chain
        pass

    def spider_opened(self, spider):
        spider.logger.info('Spider opened: %s' % spider.name)

#### setup a pipeline

In [3]:
from itemadapter import ItemAdapter

class QldlobbyistPipeline:
    def process_item(self, item, spider):
        return item

#### model for scraped items

In [4]:
class QldlobbyistItem(scrapy.Item):
    # define the fields for your item here like:
    # name = scrapy.Field()
    pass


#### Define the spider

In [5]:
from twisted.internet import reactor
import scrapy
from scrapy.crawler import CrawlerRunner
from scrapy.utils.log import configure_logging

# Reactor restart
from crochet import setup, wait_for
setup()

class QldlobbySpider(scrapy.Spider):
    name = 'qldlobbyist'
    allowed_domains = ['lobbyists.integrity.qld.gov.au']
    start_urls = ['https://lobbyists.integrity.qld.gov.au/register-details/list-lobbyists.aspx']
    custom_settings = {
        'DOWNLOAD_DELAY': '0',
        'BOT_NAME': 'qldlobbyist',
        'SPIDER_MODULES': 'qldlobbyist.spiders',
        'NEWSPIDER_MODULE': 'qldlobbyist.spiders',
        'ROBOTSTXT_OBEY': 'False',
        'FEEDS': {
            'data/qldlobbyist.csv': { # csv output
                'format': 'csv',
                'overwrite': True
            }
        }
    }
    def parse(self, response):
            url =[]
            endings = response.xpath('//*[@id="ListView"]/li/a/@href')
            for ending in endings:
                url.append('https://lobbyists.integrity.qld.gov.au/register-details/'+ending.get())

            for u in url:
                yield scrapy.Request(url=u, callback = self.parse_client_data)
    
    def parse_client_data(self, response):
        yield {
            'Lobbyist Name' : response.xpath('//*[@id="ctl00_ContentPlaceholder1_lblName"]/text()').get(),
            'Lobbying Firm' : response.xpath('//*[@id="article"]/div/table/tr[2]/td[2]//text()').get(),
            'ABN' : response.xpath('//*[@id="article"]/div/table/tr[3]/td[2]//text()').get(),
            'Position' : response.xpath('//*[@id="article"]/div/table/tr[4]/td[2]//text()').get(),
            }
        
def run_spider():
    """run spider with qldlobbyist"""
    crawler = CrawlerProcess()
    d = crawler.crawl(QldlobbySpider)
    return d

#### Start the crawler

In [6]:
run_spider()

2022-10-14 20:52:11 [scrapy.utils.log] INFO: Scrapy 2.6.3 started (bot: scrapybot)
2022-10-14 20:52:11 [scrapy.utils.log] INFO: Versions: lxml 4.6.3.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 2.0.1, Twisted 22.8.0, Python 3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 22.1.0 (OpenSSL 3.0.5 5 Jul 2022), cryptography 38.0.1, Platform Windows-10-10.0.19041-SP0
2022-10-14 20:52:11 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'qldlobbyist',
 'DOWNLOAD_DELAY': '0',
 'NEWSPIDER_MODULE': 'qldlobbyist.spiders',
 'ROBOTSTXT_OBEY': 'False',
 'SPIDER_MODULES': 'qldlobbyist.spiders'}
2022-10-14 20:52:11 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2022-10-14 20:52:11 [scrapy.extensions.telnet] INFO: Telnet Password: c24dac60b595d2d1
2022-10-14 20:52:11 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.fe

2022-10-14 20:52:12 [filelock] DEBUG: Attempting to acquire lock 2070717751248 on C:\Users\Owner\anaconda3\lib\site-packages\tldextract\.suffix_cache/publicsuffix.org-tlds\de84b5ca2167d4c83e38fb162f2e8738.tldextract.json.lock
2022-10-14 20:52:12 [filelock] INFO: Lock 2070717751248 acquired on C:\Users\Owner\anaconda3\lib\site-packages\tldextract\.suffix_cache/publicsuffix.org-tlds\de84b5ca2167d4c83e38fb162f2e8738.tldextract.json.lock
2022-10-14 20:52:12 [filelock] DEBUG: Attempting to release lock 2070717751248 on C:\Users\Owner\anaconda3\lib\site-packages\tldextract\.suffix_cache/publicsuffix.org-tlds\de84b5ca2167d4c83e38fb162f2e8738.tldextract.json.lock
2022-10-14 20:52:12 [filelock] INFO: Lock 2070717751248 released on C:\Users\Owner\anaconda3\lib\site-packages\tldextract\.suffix_cache/publicsuffix.org-tlds\de84b5ca2167d4c83e38fb162f2e8738.tldextract.json.lock
2022-10-14 20:52:12 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/l

2022-10-14 20:52:14 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=3006%20%20%20> (referer: None)
2022-10-14 20:52:14 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=864%20%20%20>
{'Lobbyist Name': 'Kyle Birks', 'Lobbying Firm': 'Gaming Machine Services', 'ABN': '63 010 353 803', 'Position': 'Manager'}
2022-10-14 20:52:14 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=3190%20%20%20> (referer: None)
2022-10-14 20:52:14 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=3006%20%20%20>
{'Lobbyist Name': 'Claire Woodwell', 'Lobbying Firm': 'CMAX Advisory', 'ABN': '73 130 740 546', 'Position': 'Government Relations Adviser'}
2022-10-14 20:52:14 [scrapy.core.engine] DEBUG: Crawl

2022-10-14 20:52:15 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=3141%20%20%20>
{'Lobbyist Name': 'Samuel Walker', 'Lobbying Firm': 'Rowland Pty Ltd', 'ABN': '59 011 033 364', 'Position': 'Advisor'}
2022-10-14 20:52:15 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=3229%20%20%20> (referer: None)
2022-10-14 20:52:15 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=3043%20%20%20>
{'Lobbyist Name': 'Elizabeth Walker', 'Lobbying Firm': 'GRACosway Pty Ltd', 'ABN': '50 082 123 822', 'Position': 'Associate'}
2022-10-14 20:52:15 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=3140%20%20%20> (referer: None)
2022-10-14 20:52:16 [scrapy.core.scraper] DEBUG: Scraped from <200 http

2022-10-14 20:52:17 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=3078%20%20%20> (referer: None)
2022-10-14 20:52:17 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=3072%20%20%20>
{'Lobbyist Name': 'Shayne Sutton', 'Lobbying Firm': 'SAS Group', 'ABN': '33 136 520 548', 'Position': 'Associate Director'}
2022-10-14 20:52:17 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=3220%20%20%20> (referer: None)
2022-10-14 20:52:17 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=3078%20%20%20>
{'Lobbyist Name': 'Edward Strong', 'Lobbying Firm': 'Nexus APAC Pty Ltd', 'ABN': '76 615 655 699', 'Position': 'Consultant'}
2022-10-14 20:52:17 [scrapy.core.engine] DEBUG: Crawled (200) <GET 

2022-10-14 20:52:18 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=3223%20%20%20>
{'Lobbyist Name': 'Adam Seaton', 'Lobbying Firm': 'Adams + Sparkes Town Planning and Development', 'ABN': '39 290 334 500', 'Position': 'Senior Town Planner'}
2022-10-14 20:52:18 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=2732%20%20%20> (referer: None)
2022-10-14 20:52:18 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=2359%20%20%20>
{'Lobbyist Name': 'Jamin Smith', 'Lobbying Firm': 'SEC Newgate', 'ABN': '38162366056', 'Position': 'Associate Partner'}
2022-10-14 20:52:18 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=2732%20%20%20>
{'Lobbyist Name': 'Jeremy Scott', 'Lobbying Firm': 'P

2022-10-14 20:52:20 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=2981%20%20%20> (referer: None)
2022-10-14 20:52:20 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=2767%20%20%20> (referer: None)
2022-10-14 20:52:20 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=3087%20%20%20>
{'Lobbyist Name': 'Geoff Rodgers', 'Lobbying Firm': 'Rowland Pty Ltd', 'ABN': '59 011 033 364', 'Position': 'Chairman'}
2022-10-14 20:52:20 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=2981%20%20%20>
{'Lobbyist Name': 'Ilana Waldman', 'Lobbying Firm': 'PremierNational', 'ABN': '71 619 450 841', 'Position': 'Director, Property Planning & Infrastructure'}
2022-10-14 20:52:20 [scrapy.core.engine]

2022-10-14 20:52:21 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=334%20%20%20>
{'Lobbyist Name': 'Damian Power', 'Lobbying Firm': 'Govstrat Pty Ltd', 'ABN': '64 964 952 044', 'Position': 'Government Relations Consultant'}
2022-10-14 20:52:21 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=3181%20%20%20> (referer: None)
2022-10-14 20:52:21 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=2845%20%20%20>
{'Lobbyist Name': 'Grace Portolesi', 'Lobbying Firm': 'Grace Portolesi Connect', 'ABN': '61 314 646 073', 'Position': 'Director'}
2022-10-14 20:52:21 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=3181%20%20%20>
{'Lobbyist Name': 'Emma Ramage', 'Lobbying Firm': 'Hawker Br

2022-10-14 20:52:23 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=869%20%20%20> (referer: None)
2022-10-14 20:52:23 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=3113%20%20%20>
{'Lobbyist Name': 'Avanti Oberoi', 'Lobbying Firm': 'CT Research Strategies Results', 'ABN': '58 101 934 454', 'Position': 'Consultant'}
2022-10-14 20:52:23 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=3131%20%20%20> (referer: None)
2022-10-14 20:52:23 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=3191%20%20%20> (referer: None)
2022-10-14 20:52:23 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=869%20%20%20>
{'

#### Read the output

In [7]:
lobbyist = pd.read_csv('data/qldlobbyist.csv')
lobbyist.head()

,Lobbyist Name,Lobbying Firm,ABN,Position
0,Karly Abbott,The Inner Circle Strategic Advisory,75634265311,Partner
1,Anthony Benscher,Barton Deakin,65 140 067 287,Managing Director
2,Paul Bigsby-Chamberlin,TG Public Affairs,23630677673,Principal / Consultant
3,Sara Benallack,BBS Communications Group Pty Ltd,34 010 899 779,Account Manager
4,Darrin Barnett,PremierNational,71 619 450 841,Labor & Crossbench Engagement


2022-10-14 20:52:24 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=3224%20%20%20>
{'Lobbyist Name': 'Juan Paolo Mendoza (Paul) Mills', 'Lobbying Firm': 'Australian Public Affairs', 'ABN': '20 098 705 403', 'Position': 'Consultant'}
2022-10-14 20:52:24 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=2681%20%20%20> (referer: None)
2022-10-14 20:52:24 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=3239%20%20%20> (referer: None)
2022-10-14 20:52:24 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=2681%20%20%20>
{'Lobbyist Name': 'Stephen Milgate', 'Lobbying Firm': 'S A Milgate & Associates Pty Ltd', 'ABN': '54 078 858 401', 'Position': 'Principal and Director'}
2022-10-14 20

2022-10-14 20:52:26 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=783%20%20%20>
{'Lobbyist Name': 'Gabriel McDowell', 'Lobbying Firm': 'Res Publica Pty Ltd', 'ABN': '63 103 178 258', 'Position': 'Managing Director'}
2022-10-14 20:52:26 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=2937%20%20%20> (referer: None)
2022-10-14 20:52:26 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=3192%20%20%20>
{'Lobbyist Name': 'Sabrine Navarro', 'Lobbying Firm': 'Michelson Alexander Pty Ltd', 'ABN': '50 660 359 866', 'Position': 'Communications Consultant'}
2022-10-14 20:52:26 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=2809%20%20%20> (referer: None)
2022-10-14 20:52:26 [scrapy.co

2022-10-14 20:52:27 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=3059%20%20%20> (referer: None)
2022-10-14 20:52:27 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=295%20%20%20>
{'Lobbyist Name': 'Peta Lange', 'Lobbying Firm': 'Ogilvy PR Agency Pty Limited', 'ABN': '89 096 965 794', 'Position': 'Head of Public Affairs'}
2022-10-14 20:52:27 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=3059%20%20%20>
{'Lobbyist Name': 'Kacey Lam', 'Lobbying Firm': 'PYNE AND PARTNERS', 'ABN': '81 634 747 363', 'Position': 'Partner'}
2022-10-14 20:52:27 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=3183%20%20%20> (referer: None)
2022-10-14 20:52:27 [scrapy.core.engine] DEBUG: Crawled 

2022-10-14 20:52:29 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=2597%20%20%20> (referer: None)
2022-10-14 20:52:29 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=3058%20%20%20>
{'Lobbyist Name': 'Andrew Joyce', 'Lobbying Firm': 'PYNE AND PARTNERS', 'ABN': '81 634 747 363', 'Position': 'Partner'}
2022-10-14 20:52:29 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=2597%20%20%20>
{'Lobbyist Name': 'Cameron Jones', 'Lobbying Firm': 'CMAX Advisory', 'ABN': '73 130 740 546', 'Position': 'Government Relations Adviser'}
2022-10-14 20:52:29 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=3210%20%20%20> (referer: None)
2022-10-14 20:52:29 [scrapy.core.engine] DEBUG: Crawled (2

2022-10-14 20:52:30 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=3251%20%20%20>
{'Lobbyist Name': 'Sophia Heather', 'Lobbying Firm': 'London Agency', 'ABN': '73145352147', 'Position': 'Public Relations Manager'}
2022-10-14 20:52:30 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=2525%20%20%20> (referer: None)
2022-10-14 20:52:30 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=2690%20%20%20>
{'Lobbyist Name': 'George Hazim', 'Lobbying Firm': 'Media and Public Affairs', 'ABN': '49622979177', 'Position': 'Director'}
2022-10-14 20:52:30 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=1094%20%20%20> (referer: None)
2022-10-14 20:52:30 [scrapy.core.scraper] DEBUG: Scraped fr

### Extract Lobbyist Client Data

This data consists of third party clients of the Lobbyist that currently retain the services of the business to provide paid or unpaid lobbyist services. \
Extracting the QLD lobbysit clients data from https://lobbyists.integrity.qld.gov.au/register-details/list-clients.aspx

####  The process for scraping the lobbyist client data uses the existing settings, and classes from the lobbyist data scraping process

#### Define the spider

In [8]:
from scrapy.crawler import CrawlerProcess

# Reactor restart
from crochet import setup, wait_for
setup()

class QldlobClientSpider(scrapy.Spider):
    name = 'qldlobbyclient'
    allowed_domains = ['lobbyists.integrity.qld.gov.au']
    start_urls = ['https://lobbyists.integrity.qld.gov.au/register-details/list-clients.aspx']
    custom_settings = {
                'DOWNLOAD_DELAY': '0',
        'BOT_NAME': 'qldlobbyist',
        'SPIDER_MODULES': 'qldlobbyist.spiders',
        'NEWSPIDER_MODULE': 'qldlobbyist.spiders',
        'ROBOTSTXT_OBEY': 'False',
        'FEEDS': {
            'data/qldlbClient.csv': { # csv output
                'format': 'csv',
                'overwrite': True
            }
        }
    }
    def parse(self, response):
            url =[]
            endings = response.xpath('//*[@id="ListView"]/li/a/@href')
            for ending in endings:
                url.append('https://lobbyists.integrity.qld.gov.au/register-details/'+ending.get())

            for u in url:
                yield scrapy.Request(url=u, callback = self.parse_client_data)
    
    def parse_client_data(self, response):
        yield {
            'Client Name' : response.xpath('//*[@id="ctl00_ContentPlaceholder1_lblCName"]/text()').get(),
            'Lobbyist' : response.xpath('//*[@id="article"]/div/table/tr[2]/td[2]//text()').get(),
            'ABN' : response.xpath('//*[@id="article"]/div/table/tr[3]/td[2]//text()').get(),
            }

def run_spider():
    """run spider with qldlobbyclient"""
    crawler = CrawlerProcess()
    d = crawler.crawl(QldlobClientSpider)
    return d

2022-10-14 20:52:30 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=3237%20%20%20>
{'Lobbyist Name': 'Winston Harris', 'Lobbying Firm': 'SAS Group', 'ABN': '33 136 520 548', 'Position': 'Senior Consultant'}
2022-10-14 20:52:30 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=2662%20%20%20> (referer: None)
2022-10-14 20:52:30 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=3174%20%20%20>
{'Lobbyist Name': 'Daniel Hoare', 'Lobbying Firm': 'Cornerstone Group', 'ABN': '9915 3936 719', 'Position': 'Director'}
2022-10-14 20:52:30 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=2456%20%20%20> (referer: None)
2022-10-14 20:52:30 [scrapy.core.scraper] DEBUG: Scraped from <200 https

2022-10-14 20:52:32 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=3160%20%20%20> (referer: None)
2022-10-14 20:52:32 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=2804%20%20%20>
{'Lobbyist Name': 'Anthony Fisk', 'Lobbying Firm': 'CGM Communications Pty Ltd', 'ABN': '66 619 642 858', 'Position': 'Executive Director'}
2022-10-14 20:52:32 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=3225%20%20%20> (referer: None)
2022-10-14 20:52:32 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=3160%20%20%20>
{'Lobbyist Name': 'Amanda Firth', 'Lobbying Firm': 'BBS Communications Group Pty Ltd', 'ABN': '34 010 899 779', 'Position': 'Group Client Manager'}
2022-10-14 20:52:32 [scrapy.

2022-10-14 20:52:33 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=3132%20%20%20>
{'Lobbyist Name': 'Jack de Hennin', 'Lobbying Firm': 'PYNE AND PARTNERS', 'ABN': '81 634 747 363', 'Position': 'Associate'}
2022-10-14 20:52:33 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=2808%20%20%20> (referer: None)
2022-10-14 20:52:33 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=2644%20%20%20> (referer: None)
2022-10-14 20:52:33 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=2896%20%20%20>
{'Lobbyist Name': 'Keren Davies', 'Lobbying Firm': 'Cornerstone Group', 'ABN': '9915 3936 719', 'Position': 'Associate Director'}
2022-10-14 20:52:33 [scrapy.core.scraper] DEBUG: Scraped from 

2022-10-14 20:52:35 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=3259%20%20%20> (referer: None)
2022-10-14 20:52:35 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=3242%20%20%20>
{'Lobbyist Name': 'Joshua Copland', 'Lobbying Firm': 'CMAX Advisory', 'ABN': '73 130 740 546', 'Position': 'Government Relations Adviser'}
2022-10-14 20:52:35 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=488%20%20%20> (referer: None)
2022-10-14 20:52:35 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=3259%20%20%20>
{'Lobbyist Name': 'Timothy Cook', 'Lobbying Firm': 'The Civic Partnership', 'ABN': '71652574171', 'Position': 'Senior Account Executive'}
2022-10-14 20:52:35 [scrapy.core.engine]

2022-10-14 20:52:36 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=706%20%20%20> (referer: None)
2022-10-14 20:52:36 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=2786%20%20%20>
{'Lobbyist Name': 'Michael Cairnduff', 'Lobbying Firm': 'Cannings Purple', 'ABN': '37 108 802 366', 'Position': 'Director'}
2022-10-14 20:52:36 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=2739%20%20%20> (referer: None)
2022-10-14 20:52:36 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=3088%20%20%20>
{'Lobbyist Name': 'Troy Collings', 'Lobbying Firm': 'Social Impact (Qld)', 'ABN': '13 112 982 323', 'Position': 'Director'}
2022-10-14 20:52:36 [scrapy.core.scraper] DEBUG: Scraped from <200 ht

2022-10-14 20:52:37 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=2807%20%20%20> (referer: None)
2022-10-14 20:52:37 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=3243%20%20%20>
{'Lobbyist Name': 'Joshua Copland', 'Lobbying Firm': 'CMAX Advisory', 'ABN': '73 130 740 546', 'Position': 'Government Relations Adviser'}
2022-10-14 20:52:37 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=2883%20%20%20> (referer: None)
2022-10-14 20:52:37 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=2807%20%20%20>
{'Lobbyist Name': 'Antony Anisse', 'Lobbying Firm': 'PremierNational', 'ABN': '71 619 450 841', 'Position': 'Special Counsel'}
2022-10-14 20:52:37 [scrapy.core.scraper] DEBUG: S

#### Start the crawler

In [9]:
run_spider()

2022-10-14 20:52:39 [scrapy.utils.log] INFO: Scrapy 2.6.3 started (bot: scrapybot)
2022-10-14 20:52:39 [scrapy.utils.log] INFO: Versions: lxml 4.6.3.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 2.0.1, Twisted 22.8.0, Python 3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 22.1.0 (OpenSSL 3.0.5 5 Jul 2022), cryptography 38.0.1, Platform Windows-10-10.0.19041-SP0
2022-10-14 20:52:39 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'qldlobbyist',
 'DOWNLOAD_DELAY': '0',
 'NEWSPIDER_MODULE': 'qldlobbyist.spiders',
 'ROBOTSTXT_OBEY': 'False',
 'SPIDER_MODULES': 'qldlobbyist.spiders'}
2022-10-14 20:52:39 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2022-10-14 20:52:39 [scrapy.extensions.telnet] INFO: Telnet Password: 2b1ab6ccd545dc57
2022-10-14 20:52:39 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.fe

2022-10-14 20:52:39 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=3253%20%20%20> (referer: None)
2022-10-14 20:52:39 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=2306%20%20%20> (referer: None)
2022-10-14 20:52:39 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=3253%20%20%20>
{'Lobbyist Name': 'Aisling Acton', 'Lobbying Firm': 'Michelson Alexander Pty Ltd', 'ABN': '50 660 359 866', 'Position': 'Senior Associate'}
2022-10-14 20:52:39 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=2836%20%20%20> (referer: None)
2022-10-14 20:52:39 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/lobbyist-details.aspx?id=2306%20%20%2

2022-10-14 20:52:41 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/client-details.aspx?id=7508%20%20%20>
{'Client Name': 'Zscaler', 'Lobbyist': 'Hawker Britton', 'ABN': '79 109 681 405'}
2022-10-14 20:52:41 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/client-details.aspx?id=6647%20%20%20> (referer: None)
2022-10-14 20:52:41 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/client-details.aspx?id=4193%20%20%20>
{'Client Name': 'Zoetis Australia Pty Ltd', 'Lobbyist': 'Willard Public Affairs Pty Ltd', 'ABN': '17 165 851 856'}
2022-10-14 20:52:41 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/client-details.aspx?id=6727%20%20%20> (referer: None)
2022-10-14 20:52:41 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/clie

2022-10-14 20:52:42 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/client-details.aspx?id=7471%20%20%20> (referer: None)
2022-10-14 20:52:42 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/client-details.aspx?id=7440%20%20%20>
{'Client Name': 'Alcohol and Drug Foundation', 'Lobbyist': 'Michelson Alexander Pty Ltd', 'ABN': '50 660 359 866'}
2022-10-14 20:52:42 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/client-details.aspx?id=6685%20%20%20>
{'Client Name': 'Yabbr Pty Ltd', 'Lobbyist': 'Anacta Strategies Pty Ltd', 'ABN': '64 633 978 677'}
2022-10-14 20:52:42 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/client-details.aspx?id=7044%20%20%20> (referer: None)
2022-10-14 20:52:42 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/regi

2022-10-14 20:52:43 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/client-details.aspx?id=7510%20%20%20> (referer: None)
2022-10-14 20:52:43 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/client-details.aspx?id=7202%20%20%20>
{'Client Name': 'Wanless Waste Management Pty Ltd', 'Lobbyist': 'Anacta Strategies Pty Ltd', 'ABN': '64 633 978 677'}
2022-10-14 20:52:43 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/client-details.aspx?id=7282%20%20%20> (referer: None)
2022-10-14 20:52:43 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/client-details.aspx?id=7510%20%20%20>
{'Client Name': 'Wambo Cattle Company', 'Lobbyist': 'PolicyWonks', 'ABN': '50 463 070 316'}
2022-10-14 20:52:43 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register

...

2022-10-14 20:53:06 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/client-details.aspx?id=7448%20%20%20>
{'Client Name': 'Platinum on Duporth Pty Ltd', 'Lobbyist': 'Precise Hospitality Licensing', 'ABN': '46220117164'}
2022-10-14 20:53:06 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/client-details.aspx?id=6959%20%20%20> (referer: None)
2022-10-14 20:53:06 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/client-details.aspx?id=7049%20%20%20>
{'Client Name': 'PLATEAU PLANTS', 'Lobbyist': 'Walsh Stevens', 'ABN': '18084661441'}
2022-10-14 20:53:06 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/client-details.aspx?id=7071%20%20%20> (referer: None)
2022-10-14 20:53:06 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/cl

2022-10-14 20:53:07 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/client-details.aspx?id=6932%20%20%20> (referer: None)
2022-10-14 20:53:07 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/client-details.aspx?id=6915%20%20%20>
{'Client Name': 'PEP Services Pty Limited', 'Lobbyist': 'GRACosway Pty Ltd', 'ABN': '50 082 123 822'}
2022-10-14 20:53:07 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/client-details.aspx?id=2597%20%20%20>
{'Client Name': 'Pine Rivers Memorial Bowls Club', 'Lobbyist': 'DWS Hospitality Specialists', 'ABN': '50 113 985 247'}
2022-10-14 20:53:07 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/client-details.aspx?id=7161%20%20%20> (referer: None)
2022-10-14 20:53:07 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.

2022-10-14 20:53:08 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/client-details.aspx?id=5661%20%20%20> (referer: None)
2022-10-14 20:53:09 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/client-details.aspx?id=7476%20%20%20>
{'Client Name': 'P. & B. BULTE (NO 2) PTY. LTD', 'Lobbyist': 'CMAX Advisory', 'ABN': '73 130 740 546'}
2022-10-14 20:53:09 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/client-details.aspx?id=6986%20%20%20> (referer: None)
2022-10-14 20:53:09 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/client-details.aspx?id=5661%20%20%20>
{'Client Name': 'Ovarian Cancer Australia Limited', 'Lobbyist': 'GRACosway Pty Ltd', 'ABN': '50 082 123 822'}
2022-10-14 20:53:09 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/regis

2022-10-14 20:53:10 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/client-details.aspx?id=6696%20%20%20>
{'Client Name': 'Nufarm Australia', 'Lobbyist': 'CMAX Advisory', 'ABN': '73 130 740 546'}
2022-10-14 20:53:10 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/client-details.aspx?id=4017%20%20%20> (referer: None)
2022-10-14 20:53:10 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/client-details.aspx?id=7307%20%20%20>
{'Client Name': 'Pacific Energy Group', 'Lobbyist': 'PolicyWonks', 'ABN': '50 463 070 316'}
2022-10-14 20:53:10 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/client-details.aspx?id=3499%20%20%20> (referer: None)
2022-10-14 20:53:10 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/client-details.aspx

2022-10-14 20:53:11 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/client-details.aspx?id=6813%20%20%20> (referer: None)
2022-10-14 20:53:11 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/client-details.aspx?id=847%20%20%20>
{'Client Name': 'Northern Investment Holdings Pty Ltd', 'Lobbyist': 'Commercial Licensing Specialists', 'ABN': '33 134 318 595'}
2022-10-14 20:53:11 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/client-details.aspx?id=5869%20%20%20>
{'Client Name': 'Nerang RSL', 'Lobbyist': 'Promedia Public Relations', 'ABN': '52 010 453 826'}
2022-10-14 20:53:11 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/client-details.aspx?id=6164%20%20%20> (referer: None)
2022-10-14 20:53:11 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.g

#### Read the output

In [10]:
client = pd.read_csv('data/qldlbClient.csv')
client.head()

,Client Name,Lobbyist,ABN
0,1st Group,TG Public Affairs,23630677673
1,Amazon,Hawker Britton,79 109 681 405
2,All About the Food Pty Ltd,Commercial Licensing Specialists,33 134 318 595
3,Amicus Therapeutics,Ogilvy PR Agency Pty Limited,89 096 965 794
4,Alexandra Community Group,Project Urban Pty Ltd,97 608 895 923


2022-10-14 20:53:12 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/client-details.aspx?id=7231%20%20%20>
{'Client Name': 'NEP Broadcast Services', 'Lobbyist': 'Next Level Strategic Services', 'ABN': '27 613 857 668'}
2022-10-14 20:53:12 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/client-details.aspx?id=1041%20%20%20> (referer: None)
2022-10-14 20:53:12 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/client-details.aspx?id=6347%20%20%20>
{'Client Name': 'New Hope Group', 'Lobbyist': 'Momentum Strategic Communication Services', 'ABN': '66 196 172 628'}
2022-10-14 20:53:12 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/client-details.aspx?id=846%20%20%20> (referer: None)
2022-10-14 20:53:12 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.ql

2022-10-14 20:53:13 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/client-details.aspx?id=6525%20%20%20> (referer: None)
2022-10-14 20:53:13 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/client-details.aspx?id=7422%20%20%20>
{'Client Name': 'Moggill Cafe Pty Ltd', 'Lobbyist': 'Liquor Licensing Consultants', 'ABN': '58 679 042 968'}
2022-10-14 20:53:13 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/client-details.aspx?id=6526%20%20%20> (referer: None)
2022-10-14 20:53:13 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/client-details.aspx?id=6525%20%20%20>
{'Client Name': 'Metro Trains Melbourne', 'Lobbyist': 'Conlon Advisory', 'ABN': '11 951 391 072'}
2022-10-14 20:53:13 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-de

2022-10-14 20:53:14 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/client-details.aspx?id=6065%20%20%20> (referer: None)
2022-10-14 20:53:14 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/client-details.aspx?id=4594%20%20%20>
{'Client Name': 'Mars Petcare Australia', 'Lobbyist': 'FPL Advisory', 'ABN': '34123819385'}
2022-10-14 20:53:15 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/client-details.aspx?id=6860%20%20%20> (referer: None)
2022-10-14 20:53:15 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/client-details.aspx?id=7019%20%20%20> (referer: None)
2022-10-14 20:53:15 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/client-details.aspx?id=6065%20%20%20>
{'Client Name': 'MARS Australia Pty Ltd', 'Lobbyist': '

2022-10-14 20:53:16 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/client-details.aspx?id=7002%20%20%20> (referer: None)
2022-10-14 20:53:16 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/client-details.aspx?id=6919%20%20%20>
{'Client Name': 'Magia Solutions Pty Ltd', 'Lobbyist': 'Hawker Britton', 'ABN': '79 109 681 405'}
2022-10-14 20:53:16 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/client-details.aspx?id=7105%20%20%20> (referer: None)
2022-10-14 20:53:16 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/client-details.aspx?id=7002%20%20%20>
{'Client Name': 'M Resources', 'Lobbyist': 'CRISIS&COMMS CO', 'ABN': '56 637 786 899'}
2022-10-14 20:53:16 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/client-details.a

...

2022-10-14 20:53:54 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/client-details.aspx?id=6782%20%20%20> (referer: None)
2022-10-14 20:53:54 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/client-details.aspx?id=6545%20%20%20>
{'Client Name': 'Australian Chiropractors Association', 'Lobbyist': 'Ogilvy PR Agency Pty Limited', 'ABN': '89 096 965 794'}
2022-10-14 20:53:54 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/client-details.aspx?id=6989%20%20%20> (referer: None)
2022-10-14 20:53:54 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/client-details.aspx?id=6782%20%20%20>
{'Client Name': 'Australian Automotive Dealer Association', 'Lobbyist': 'PremierNational', 'ABN': '71 619 450 841'}
2022-10-14 20:53:54 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyist

2022-10-14 20:53:55 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/client-details.aspx?id=7095%20%20%20>
{'Client Name': 'Ateco Group', 'Lobbyist': 'Iemma Patterson Premier Advisory', 'ABN': '53 830 148 273'}
2022-10-14 20:53:55 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/client-details.aspx?id=7529%20%20%20> (referer: None)
2022-10-14 20:53:55 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/client-details.aspx?id=7048%20%20%20>
{'Client Name': 'ASYS COMPUTER CONSULTING PTY. LTD.', 'Lobbyist': 'Walsh Stevens', 'ABN': '18084661441'}
2022-10-14 20:53:55 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/client-details.aspx?id=82%20%20%20> (referer: None)
2022-10-14 20:53:55 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-de

2022-10-14 20:53:56 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/client-details.aspx?id=6443%20%20%20> (referer: None)
2022-10-14 20:53:56 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/client-details.aspx?id=7116%20%20%20>
{'Client Name': 'ANDHealth', 'Lobbyist': 'CPR Communications and Public Relations Pty Ltd (Melbourne)', 'ABN': 'ABN 94064357544, ACN 064357544'}
2022-10-14 20:53:56 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/client-details.aspx?id=7489%20%20%20> (referer: None)
2022-10-14 20:53:56 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/client-details.aspx?id=6443%20%20%20>
{'Client Name': 'Aircraft Owners and Pilots Association of Australia', 'Lobbyist': 'Strategic Political Counsel', 'ABN': '24613884763'}
2022-10-14 20:53:56 [scrapy.core.engine] DE

2022-10-14 20:53:58 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/client-details.aspx?id=6569%20%20%20> (referer: None)
2022-10-14 20:53:58 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/client-details.aspx?id=7382%20%20%20>
{'Client Name': 'ADG Engineering Pty Ltd', 'Lobbyist': 'Rowland Pty Ltd', 'ABN': '59 011 033 364'}
2022-10-14 20:53:58 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/register-details/client-details.aspx?id=7364%20%20%20> (referer: None)
2022-10-14 20:53:58 [scrapy.core.scraper] DEBUG: Scraped from <200 https://lobbyists.integrity.qld.gov.au/register-details/client-details.aspx?id=6569%20%20%20>
{'Client Name': 'Annsca Energy PTY LTD', 'Lobbyist': 'S A Milgate & Associates Pty Ltd', 'ABN': '54 078 858 401'}
2022-10-14 20:53:58 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://lobbyists.integrity.qld.gov.au/regis

2022-10-14 20:54:00 [scrapy.core.engine] INFO: Spider closed (finished)


### Extract Donations Data

A disclosure return is the reporting of all donations, loans, and expenditure incurred for an election campaign. 
These must be reported to the ECQ under the Electoral Act 1992 and the Local Government Electoral Act 2011. 
All disclosures are made through the Electronic Disclosure System (EDS) and are available to the public.

This data is scraped from the EDS (https://disclosures.ecq.qld.gov.au/Map)

The process below scrapes the source for new data, then appends to the existing data at '**data/qld_donations.csv**'.

In [33]:
# Import Packages
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import re
import pandas as pd
import datetime

# Define the browser to use and then open up the donations homepage
driver = webdriver.Chrome()
driver.get('https://disclosures.ecq.qld.gov.au/Map')

# Empty List to collect new urls
recipient_url = []

# Read the latest donations csv and determine latest date
df_date = pd.DataFrame()
df_donations = pd.read_csv('data/qld_donations.csv')
df_date['date'] = df_donations.date.dropna()
df_date['date'] = pd.to_datetime(df_date['date'], format='%d/%m/%Y')
last_date = max(df_date.date)
newest_date = (last_date.date().strftime('%d-%m-%Y'))

# Input the last date into the website, then apply filter
date_from = driver.find_element(By.XPATH, "//*[@id='ViewFilter_DateFrom']")
date_from.send_keys(newest_date)
# date_from.send_keys('01-09-2022') # for manual filter
apply_button = driver.find_element(By.XPATH, "//*[@id='maps-form']/div/div/div[2]/div[1]/button")
apply_button.click()
time.sleep(2)

# 2. Begin scraping the urls
while True:
    
     # This code selects all the URLs from the main table, then appends URLs to a list
    recip_urls = driver.find_elements(By.XPATH, "//tr/td[3]/a")
    for url in recip_urls:
        #print(url.get_attribute("href"))
        recipient_url.append(url.get_attribute("href"))
    
    # for current page, grab the page items at bottom
    page_items_element = driver.find_element(By.XPATH, "//*[@id='map-page-header']/div[2]/div[3]/div/div/div/div/div/div[2]/small")
    page_items = page_items_element.text
    page_item_nos = re.findall(r'\d+', page_items)
    
    # pagination if page items are not at the end
    if page_item_nos[1] != page_item_nos[2]:
        # looks for the next button
        button = driver.find_element(By.CLASS_NAME, "fa.fa-chevron-right")
        button.click()
        time.sleep(2)
    
    else:
        # exits when the button no longer exists
        break

# Instantiate all lists        
all_urls = recipient_url
date = []
donor_name = []
donor_type = []
recipient_name = []
recipient_type = []
agent_names = []
gift_type = []
gift_value = []
page_url= []


# Scrape the pages from each of the just scraped urls
for url in all_urls:
    driver.get(url)
    
    # Donor
    try: 
        driver.find_element(By.XPATH, "//*[@class='form-group']/div/input")
    except NoSuchElementException:
        donor_name.append(None)
    else:
        donor_dim = driver.find_element(By.XPATH, "//*[@class='form-group']/div/input")
        donor_name.append(donor_dim.get_attribute("value"))
    
    
    # Donor_Types
    try: 
        driver.find_element(By.XPATH, "//*[@id='disclosureEntries']/div/div/div[1]/div/span[4]")
    except NoSuchElementException:
        donor_type.append(None)
    else:
        donor_type_dim = driver.find_element(By.XPATH, "//*[@id='disclosureEntries']/div/div/div[1]/div/span[4]")
        donor_type.append(donor_type_dim.text)

    # Recipient
    try:
        driver.find_element(By.XPATH, "//*[@id='Head_ElectorFullName']")
    except NoSuchElementException:
        try:
            driver.find_element(By.XPATH, "//*[@id='Head_PoliticalPartyTitle']")
        except NoSuchElementException:
            try:
                driver.find_element(By.XPATH, "//*[@id='Head_Title']")
            except NoSuchElementException:
                recipient_name.append(None)
            else:
                recipient_dim = driver.find_element(By.XPATH, "//*[@id='Head_Title']")
                recipient_name.append(recipient_dim.get_attribute("value"))
        else:
            recipient_dim = driver.find_element(By.XPATH, "//*[@id='Head_PoliticalPartyTitle']")
            recipient_name.append(recipient_dim.get_attribute("value"))
    else:
        recipient_dim = driver.find_element(By.XPATH, "//*[@id='Head_ElectorFullName']")
        recipient_name.append(recipient_dim.get_attribute("value"))    
    
    
    
    # Recipient Type
    try:
        driver.find_element(By.XPATH, "//*[@id='content']/div/div[1]/div/div/div/div/div[1]/h1")
    except NoSuchElementException:
        recipient_type.append(None)
    else:
        recip_type_dim = driver.find_element(By.XPATH, "//*[@id='content']/div/div[1]/div/div/div/div/div[1]/h1")
        recipient_type.append(recip_type_dim.text)
    
    # Date
    try:
        driver.find_element(By.XPATH, "//*[@class='form-control datepicker gift-date-received special-reporting-event-check-trigger']")
    except NoSuchElementException:
        date.append(None)
    else:
        date_dim = driver.find_element(By.XPATH, "//*[@class='form-control datepicker gift-date-received special-reporting-event-check-trigger']")
        date.append(date_dim.get_attribute("value"))
    
    # Gift value
    try:
        driver.find_element(By.XPATH, "//*[@class='form-control currencyFormat text-right gift-amount special-reporting-event-check-trigger']")
    except NoSuchElementException:
        gift_value.append(None)
    else:
        gift_value_dim = driver.find_element(By.XPATH, "//*[@class='form-control currencyFormat text-right gift-amount special-reporting-event-check-trigger']")
        gift_value.append(gift_value_dim.get_attribute("value"))
    
    # Gift Type
    try:
        driver.find_element(By.XPATH, "//*[@id='disclosureEntries']/div/div/div[1]/div/span[2]")
    except NoSuchElementException:
        gift_type.append(None)
    else:
        gift_type_dim = driver.find_element(By.XPATH, "//*[@id='disclosureEntries']/div/div/div[1]/div/span[2]")
        gift_type.append(gift_type_dim.text)
    
    # Agent Names
    try:
        driver.find_element(By.XPATH, "//*[@id='Head_RepresentativeFullName']")
    except NoSuchElementException:
        try: 
            driver.find_element(By.XPATH, "//*[@id='Head_AgentFullName']")
        except NoSuchElementException:
            agent_names.append(None)
        else:
            agent_name = driver.find_element(By.XPATH, "//*[@id='Head_AgentFullName']")
            agent_names.append(agent_name.get_attribute("value"))   
    else:
        agent_name = driver.find_element(By.XPATH, "//*[@id='Head_RepresentativeFullName']")
        agent_names.append(agent_name.get_attribute("value"))
        
        
    # original url
    page_url.append(driver.current_url)
        
    time.sleep(0.5)
    
    
# Create dataframe for the newly scraped pages
df = pd.DataFrame(list(zip(date, donor_name, donor_type, recipient_name, recipient_type, agent_names, gift_type, gift_value, page_url)), 
                  columns = ["date", "donor_name", "donor_type", "recipient_name", "recipient_type", "agent_names", "gift_type", "gift_value", "page_url"])

# Clean any whitespaces
df['recipient_name'] = df['recipient_name'].str.strip()
df['donor_name'] = df['donor_name'].str.strip()
df['agent_names'] = df['agent_names'].str.strip()

# Import existing data, concatenate the new data with existing dataset, remove duplicates
df_concat = pd.concat([df, df_donations], ignore_index=True)
df_dupes_dropped = df_concat.drop_duplicates(subset=['page_url'])
df_dupes_dropped = df_dupes_dropped[df_dupes_dropped['gift_value'].notna()]

# Save new file
df_dupes_dropped.to_csv('data/qld_donations.csv', header=True, index=False)

# close the browser
driver.quit()

SyntaxError: invalid syntax (<ipython-input-33-38cbb792515d>, line 192)

In [13]:
donations = df_dupes_dropped
donations.head()

,date,donor_name,donor_type,recipient_name,recipient_type,agent_names,gift_type,gift_value,page_url
0,7/10/2022,Ellen MacQueen,an individual,Queensland Greens,Agent for political party,SUSAN ETHERIDGE,a Gift,1000.00,https://disclosures.ecq.qld.gov.au/forms/recip...
2,6/10/2022,JENNIFER ANNE HORSBURGH,an individual,Animal Justice Party (Queensland),Agent for political party,Lindon Cox,a Gift,300.0,https://disclosures.ecq.qld.gov.au/forms/recip...
3,5/10/2022,JONATHAN SRIRANGANATHAN,an individual,Queensland Greens,Agent for political party,SUSAN ETHERIDGE,a Gift,5177.0,https://disclosures.ecq.qld.gov.au/forms/recip...
4,30/09/2022,AUSTRALIAN GREENS THE GREENS INCORPORATED,a corporation,Queensland Greens,Agent for political party,SUSAN ETHERIDGE,a Gift,185440.0,https://disclosures.ecq.qld.gov.au/forms/recip...
5,30/09/2022,Roger Welch,an individual,Liberal National Party of Queensland,Agent for political party,MICHAEL NEGEREVICH,a Gift,1000.0,https://disclosures.ecq.qld.gov.au/forms/recip...


# Pre-processing
To extract 'source', 'target', and 'weight' columns

In [55]:
# lobbyist
ag_lobbyist = lobbyist.groupby(['Lobbying Firm','Lobbyist Name']).size().reset_index(name='edge weight')
ag_lobbyist = ag_lobbyist.rename(columns={'Lobbying Firm': 'source', 'Lobbyist Name': 'target'})
ag_lobbyist['node weight'] = '1' # default weight
ag_lobbyist['color'] = '#BEBEBE'
ag_lobbyist['source type'] = 'Lobbying Firm'
ag_lobbyist['target type'] = 'Lobbyist'
ag_lobbyist.head()

,source,target,edge weight,node weight,color,source type,target type
0,Adams + Sparkes Town Planning and Development,Adam Seaton,1,1,#BEBEBE,Lobbying Firm,Lobbyist
1,Adams + Sparkes Town Planning and Development,Peter Sparkes,1,1,#BEBEBE,Lobbying Firm,Lobbyist
2,Anacta Strategies Pty Ltd,Jessame Tibbitts,1,1,#BEBEBE,Lobbying Firm,Lobbyist
3,Anacta Strategies Pty Ltd,Liam O'Regan,1,1,#BEBEBE,Lobbying Firm,Lobbyist
4,Atlas Alliance,Douglas Pye,1,1,#BEBEBE,Lobbying Firm,Lobbyist


In [56]:
# client
ag_client = client.groupby(['Lobbyist','Client Name']).size().reset_index(name='edge weight')
ag_client = ag_client.rename(columns={'Lobbyist': 'source', 'Client Name': 'target'})
ag_client['node weight'] = '1' # default weight
ag_client['color'] = '#BEBEBE'
ag_client['source type'] = 'Lobbyist'
ag_client['target type'] = 'Client'
ag_client.head()

,source,target,edge weight,node weight,color,source type,target type
0,Adams + Sparkes Town Planning and Development,W & M Carnall Pty Ltd,1,1,#BEBEBE,Lobbyist,Client
1,Anacta Strategies Pty Ltd,Pinssar Pty Ltd,1,1,#BEBEBE,Lobbyist,Client
2,Anacta Strategies Pty Ltd,Plenary Group,1,1,#BEBEBE,Lobbyist,Client
3,Anacta Strategies Pty Ltd,QER Pty Ltd,1,1,#BEBEBE,Lobbyist,Client
4,Anacta Strategies Pty Ltd,Queensland Motorways Pty Ltd,1,1,#BEBEBE,Lobbyist,Client


In [57]:
# donations
donations["gift_value"] = pd.to_numeric(donations["gift_value"])
ag_donations = donations.groupby(['donor_name','recipient_name','donor_type','recipient_type'])['gift_value'].agg(['sum', 'count']).reset_index()
ag_donations = ag_donations.rename(columns={'donor_name': 'source', 'recipient_name': 'target',
                                            'donor_type': 'source type', 'recipient_type': 'target type', 
                                            'sum':'node weight', 'count':'edge weight'})
ag_donations['color'] = '#BEBEBE'

ag_donations = ag_donations[['source', 'target', 'edge weight', 'node weight','color',
                             'source type', 'target type']]

# transform donor type
ag_donations["source type"].replace({"a corporation": "Corporation", "an individual": "Individual",
                           "another type of entity": "Other", "other": "Other",
                           "a trust fund or foundation": "Trust/Foundation", 
                           "an unincorporated association": "Unincorporated Association"}, inplace=True)

ag_donations.head()

,source,target,edge weight,node weight,color,source type,target type
0,141 Abbott Street Pty Ltd,Australian Labor Party (State of Queensland),1,2000.0,#BEBEBE,Corporation,Agent for political party
1,188 Edward Pty Ltd,Liberal National Party of Queensland,1,2000.0,#BEBEBE,Corporation,Agent for political party
2,1st Class Food Pty Ltd,Liberal National Party of Queensland,2,12000.0,#BEBEBE,Corporation,Agent for political party
3,235L Projects Pty Ltd,Liberal National Party of Queensland,1,5500.0,#BEBEBE,Corporation,Agent for political party
4,24 Kurilpa Street West End Pty Ltd,Liberal National Party of Queensland,2,5000.0,#BEBEBE,Corporation,Agent for political party


## Stack all data and output
output qld_networkg.csv

In [58]:
map_dat = pd.concat([ag_lobbyist,ag_client,ag_donations],ignore_index=True)
map_dat.head()

,source,target,edge weight,node weight,color,source type,target type
0,Adams + Sparkes Town Planning and Development,Adam Seaton,1,1,#BEBEBE,Lobbying Firm,Lobbyist
1,Adams + Sparkes Town Planning and Development,Peter Sparkes,1,1,#BEBEBE,Lobbying Firm,Lobbyist
2,Anacta Strategies Pty Ltd,Jessame Tibbitts,1,1,#BEBEBE,Lobbying Firm,Lobbyist
3,Anacta Strategies Pty Ltd,Liam O'Regan,1,1,#BEBEBE,Lobbying Firm,Lobbyist
4,Atlas Alliance,Douglas Pye,1,1,#BEBEBE,Lobbying Firm,Lobbyist


#### Cusomise colour

In [59]:
# custom colours
map_dat.loc[map_dat['source type'] == 'Lobbying Firm', 'color'] = "#9d94ff" # purple
map_dat.loc[map_dat['source type'] == 'Lobbyist', 'color'] = "#c780e8" # purple alt
map_dat.loc[map_dat['source type'] == 'Corporation', 'color'] = "#ff6961" # red
map_dat.loc[map_dat['source type'] == 'Individual', 'color'] = "#42d6a4" # green
map_dat.loc[map_dat['source type'] == 'Other', 'color'] = "#BEBEBE" # grey 
map_dat.loc[map_dat['source type'] == 'Trust/Foundation', 'color'] = "#59adf6" # blue
map_dat.loc[map_dat['source type'] == 'Unincorporated Association', 'color'] = "#ffb480" # orange
map_dat = map_dat.rename(columns={'color': 'source color'})

map_dat.loc[map_dat['target type'] == 'Lobbyist', 'target color'] = "#c780e8" # purple alt
map_dat.loc[map_dat['target type'] == 'Client', 'target color'] = "#ff6961" # red
map_dat.loc[map_dat['target type'] == 'Agent for political party', 'target color'] = "#08cad1" # aqua
map_dat.loc[map_dat['target type'] == 'State candidate', 'target color'] = "#0565f7" # deep blue 
map_dat.loc[map_dat['target type'] == 'Agent for state candidate', 'target color'] = "#08cad1" # aqua
map_dat.loc[map_dat['target type'] == 'Organisation', 'target color'] = "#ff6961" # red

map_dat.head()

,source,target,edge weight,node weight,source color,source type,target type,target color
0,Adams + Sparkes Town Planning and Development,Adam Seaton,1,1,#9d94ff,Lobbying Firm,Lobbyist,#c780e8
1,Adams + Sparkes Town Planning and Development,Peter Sparkes,1,1,#9d94ff,Lobbying Firm,Lobbyist,#c780e8
2,Anacta Strategies Pty Ltd,Jessame Tibbitts,1,1,#9d94ff,Lobbying Firm,Lobbyist,#c780e8
3,Anacta Strategies Pty Ltd,Liam O'Regan,1,1,#9d94ff,Lobbying Firm,Lobbyist,#c780e8
4,Atlas Alliance,Douglas Pye,1,1,#9d94ff,Lobbying Firm,Lobbyist,#c780e8


In [61]:
map_dat = map_dat[['source', 'target', 'edge weight', 'node weight',
                   'source type', 'target type', 'source color', 'target color']]
map_dat.head()

,source,target,edge weight,node weight,source type,target type,source color,target color
0,Adams + Sparkes Town Planning and Development,Adam Seaton,1,1,Lobbying Firm,Lobbyist,#9d94ff,#c780e8
1,Adams + Sparkes Town Planning and Development,Peter Sparkes,1,1,Lobbying Firm,Lobbyist,#9d94ff,#c780e8
2,Anacta Strategies Pty Ltd,Jessame Tibbitts,1,1,Lobbying Firm,Lobbyist,#9d94ff,#c780e8
3,Anacta Strategies Pty Ltd,Liam O'Regan,1,1,Lobbying Firm,Lobbyist,#9d94ff,#c780e8
4,Atlas Alliance,Douglas Pye,1,1,Lobbying Firm,Lobbyist,#9d94ff,#c780e8


#### Output network data
output qld_networkg.csv

In [62]:
map_dat.to_csv(r'data\qld_networkg.csv', encoding='utf-8', index=False)

### Only run below if you wish to open the folder location.

In [63]:
# open data folder on Windows
import os
path = "data"
path = os.path.realpath(path)
os.startfile(path)

In [ ]:
# for Mac
mport subprocess
import os
path = "data"
if os.path.exists(path):
    subprocess.call(["open", path])